# Task description

In this lab 3 you will implement in python three cryptographic protocols for key exchange and digital
signature.
For the first protocol, two codes have to be developed:
• a code based on RSA and AES that you already developed during previous exercise
and lab sessions.
• a code using pycryptodome
For the two others you can use pycryptodome only.
For the hash function you can use the SHA256 algorithm as used in exercise session 2 (but other hash
algorithms are possible).
For all programs you will implement both Alice and Bob sides in order to build the whole protocol.
For each protocol you will consider a text file containing the plaintext that will be either
ciphered/deciphered or signed/verified.
For the three protocols proposed a test sequence to validate and demonstrate your programs.
All solutions must be clearly explained in the report.

In [ ]:
# @title Code for AES and RSA from Lab 2 and Exercise 3
import random, sys, os, math
def gcd(a, b):
    # Return the Greatest Common Divisor of a and b using Euclid's Algorithm
    while a != 0:
        a, b = b % a, a
    return b
def findModInverse(a, m):
    # Return the modular inverse of a % m, which is
    # the number x such that a*x % m = 1

    if gcd(a, m) != 1:
        return None # No mod inverse exists if a & m aren't relatively prime.

    # Calculate using the Extended Euclidean Algorithm:
    u1, u2, u3 = 1, 0, a
    v1, v2, v3 = 0, 1, m
    while v3 != 0:
        q = u3 // v3 # Note that // is the integer division operator
        v1, v2, v3, u1, u2, u3 = (u1 - q * v1), (u2 - q * v2), (u3 - q * v3), v1, v2, v3
    return u1 % m
def primeSieve(sieveSize):
    # Returns a list of prime numbers calculated using
    # the Sieve of Eratosthenes algorithm.

    sieve = [True] * sieveSize
    sieve[0] = False # Zero and one are not prime numbers.
    sieve[1] = False

    # Create the sieve:
    for i in range(2, int(math.sqrt(sieveSize)) + 1):
        pointer = i * 2
        while pointer < sieveSize:
            sieve[pointer] = False
            pointer += i

    # Compile the list of primes:
    primes = []
    for i in range(sieveSize):
        if sieve[i] == True:
            primes.append(i)

    return primes
LOW_PRIMES = primeSieve(100)
import random
def rabinMiller(num):
    # Returns True if num is a prime number.
    if num % 2 == 0 or num < 2:
        return False # Rabin-Miller doesn't work on even integers.
    if num == 3:
        return True
    s = num - 1
    t = 0
    while s % 2 == 0:
        # Keep halving s until it is odd (and use t
        # to count how many times we halve s):
        s = s // 2
        t += 1
    for trials in range(5): # Try to falsify num's primality 5 times.
        a = random.randrange(2, num - 1)
        v = pow(a, s, num)
        if v != 1: # (This test does not apply if v is 1.)
            i = 0
            while v != (num - 1):
                if i == t - 1:
                    return False
                else:
                    i = i + 1
                    v = (v ** 2) % num
    return True
def isPrime(num):
    # Return True if num is a prime number. This function does a quicker
    # prime number check before calling rabinMiller().
    if (num < 2):
        return False # 0, 1, and negative numbers are not prime.
    # See if any of the low prime numbers can divide num:
    for prime in LOW_PRIMES:
        if (num == prime):
            return True
    for prime in LOW_PRIMES:
        if (num % prime == 0):
            return False
    # If all else fails, call rabinMiller() to determine if num is a prime:
    return rabinMiller(num)
def generateLargePrime(keysize=1024):
    # Return a random prime number that is keysize bits in size.
    i=0
    while True:
        num = random.randrange(2**(keysize-1), 2**(keysize))
        i = i + 1
        if isPrime(num):
            return num
def generateKey(keySize):
    # Creates a public/private keys keySize bits in size.
    p = 0
    q = 0
    # Step 1: Create two prime numbers, p and q. Calculate n = p * q.
    while p == q:
        p = generateLargePrime(keySize)
        q = generateLargePrime(keySize)
    n = p * q
    phi = (p - 1) * (q - 1)
    # Step 2: Create a number e that is relatively prime to (p-1)*(q-1):
    e = 2
    while gcd(e, phi) != 1:
        # Keep trying random numbers for e until one is valid:
        e = random.randrange(2 ** (keySize - 1), 2 ** (keySize))

    # Step 3: Calculate d, the mod inverse of e:
    d = findModInverse(e, phi)

    publicKey = (n, e)
    privateKey = (n, d)

    return (publicKey, privateKey)
def makeKeyFiles(name, keySize):
    # Creates two files 'x_pubkey.txt' and 'x_privkey.txt' (where x
    # is the value in name) with the n,e and d,e integers written in
    # them, delimited by a comma.

    # Our safety check will prevent us from overwriting our old key files:
    if os.path.exists('%s_pubkey.txt' % (name)) or os.path.exists('%s_privkey.txt' % (name)):
        sys.exit('WARNING: The file %s_pubkey.txt or %s_privkey.txt already exists! Use a different name or delete these files and re-run this program.' % (name, name))

    publicKey, privateKey = generateKey(keySize)

    fo = open('%s_pubkey.txt' % (name), 'w')
    fo.write('%s,%s,%s' % (keySize, publicKey[0], publicKey[1]))
    fo.close()

    fo = open('%s_privkey.txt' % (name), 'w')
    fo.write('%s,%s,%s' % (keySize, privateKey[0], privateKey[1]))
    fo.close()
    return publicKey, privateKey
def readKeyFile(keyFilename):
    # Given the filename of a file that contains a public or private key,
    # return the key as a (n,e) or (n,d) tuple value.
    fo = open(keyFilename)
    content = fo.read()
    fo.close()
    keySize, n, EorD = content.split(',')
    return (int(keySize), int(n), int(EorD))
# IMPORTANT: The block size MUST be less than or equal to the key size!
# (Note: The block size is in bytes, the key size is in bits. There
# are 8 bits in 1 byte.)

DEFAULT_BLOCK_SIZE = 128 # 128 bytes
BYTE_SIZE = 256 # One byte has 256 different values.

def getBlocksFromText(message, blockSize=DEFAULT_BLOCK_SIZE):
    # Converts a string message to a list of block integers. Each integer
    # represents 128 (or whatever blockSize is set to) string characters.

    messageBytes = message.encode('ascii') # convert the string to bytes

    blockInts = []
    for blockStart in range(0, len(messageBytes), blockSize):
        # Calculate the block integer for this block of text
        blockInt = 0
        for i in range(blockStart, min(blockStart + blockSize, len(messageBytes))):
            blockInt += messageBytes[i] * (BYTE_SIZE ** (i % blockSize))
        blockInts.append(blockInt)
    return blockInts


def encryptMessage(message, key, blockSize=DEFAULT_BLOCK_SIZE):
    # Converts the message string into a list of block integers, and then
    # encrypts each block integer. Pass the PUBLIC key to encrypt.
    encryptedBlocks = []
    n, e = key

    for block in getBlocksFromText(message, blockSize):
        # ciphertext = plaintext ^ e mod n

        # complete the code to perform the modular exponentiation (encryption)
        # ...
        ciphertext = pow(block, e, n)
        encryptedBlocks.append(ciphertext)

    return encryptedBlocks


def readKeyFile(keyFilename):
    # Given the filename of a file that contains a public or private key,
    # return the key as a (n,e) or (n,d) tuple value.
    fo = open(keyFilename)
    content = fo.read()
    fo.close()
    keySize, n, EorD = content.split(',')
    return (int(keySize), int(n), int(EorD))
# IMPORTANT: The block size MUST be less than or equal to the key size!
# (Note: The block size is in bytes, the key size is in bits. There
# are 8 bits in 1 byte.)

DEFAULT_BLOCK_SIZE = 128 # 128 bytes
BYTE_SIZE = 256 # One byte has 256 different values.

def getTextFromBlocks(blockInts, messageLength, blockSize=DEFAULT_BLOCK_SIZE):
    # Converts a list of block integers to the original message string.
    # The original message length is needed to properly convert the last
    # block integer.
    message = []
    for blockInt in blockInts:
        blockMessage = []
        for i in range(blockSize - 1, -1, -1):
            if len(message) + i < messageLength:
                # Decode the message string for the 128 (or whatever
                # blockSize is set to) characters from this block integer.
                asciiNumber = blockInt // (BYTE_SIZE ** i)
                #print(chr(asciiNumber))
                #blockInt = blockInt % (BYTE_SIZE ** i)
                blockInt = blockInt % (pow(BYTE_SIZE, i))
                blockMessage.insert(0, chr(asciiNumber))
        message.extend(blockMessage)
    return ''.join(message)

def decryptMessage(encryptedBlocks, messageLength, key, blockSize=DEFAULT_BLOCK_SIZE):
    # Decrypts a list of encrypted block ints into the original message
    # string. The original message length is required to properly decrypt
    # the last block. Be sure to pass the PRIVATE key to decrypt.
    decryptedBlocks = []
    n, d = key
    for block in encryptedBlocks:
        # plaintext = ciphertext ^ d mod n

        # complete the code to perform the modular exponentiation (decryption)
        # ...
        plaintext = pow(block, d, n)
        decryptedBlocks.append(plaintext)
    return getTextFromBlocks(decryptedBlocks, messageLength, blockSize)

def readFromFileAndDecrypt(messageFilename, keyFilename):
    # Using a key from a key file, read an encrypted message from a file
    # and then decrypt it. Returns the decrypted message string.
    keySize, n, d = readKeyFile(keyFilename)


    # Read in the message length and the encrypted message from the file.
    fo = open(messageFilename)
    content = fo.read()
    messageLength, blockSize, encryptedMessage = content.split('_')
    messageLength = int(messageLength)
    blockSize = int(blockSize)

    # Check that key size is greater than block size.
    if keySize < blockSize * 8: # * 8 to convert bytes to bits
        sys.exit('ERROR: Block size is %s bits and key size is %s bits. The RSA cipher requires the block size to be equal to or greater than the key size. Did you specify the correct key file and encrypted file?' % (blockSize * 8, keySize))

    # Convert the encrypted message into large int values.
    encryptedBlocks = []
    for block in encryptedMessage.split(','):
        encryptedBlocks.append(int(block))

    # Decrypt the large int values.
    return decryptMessage(encryptedBlocks, messageLength, (n, d), blockSize)

"""
    Copyright (C) 2012 Bo Zhu http://about.bozhu.me

    Permission is hereby granted, free of charge, to any person obtaining a
    copy of this software and associated documentation files (the "Software"),
    to deal in the Software without restriction, including without limitation
    the rights to use, copy, modify, merge, publish, distribute, sublicense,
    and/or sell copies of the Software, and to permit persons to whom the
    Software is furnished to do so, subject to the following conditions:

    The above copyright notice and this permission notice shall be included in
    all copies or substantial portions of the Software.

    THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
    IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
    FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
    THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
    LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
    FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
    DEALINGS IN THE SOFTWARE.
"""

"""
    Modified by Guy Gogniat 2021
"""

Sbox = (
    0x63, 0x7C, 0x77, 0x7B, 0xF2, 0x6B, 0x6F, 0xC5, 0x30, 0x01, 0x67, 0x2B, 0xFE, 0xD7, 0xAB, 0x76,
    0xCA, 0x82, 0xC9, 0x7D, 0xFA, 0x59, 0x47, 0xF0, 0xAD, 0xD4, 0xA2, 0xAF, 0x9C, 0xA4, 0x72, 0xC0,
    0xB7, 0xFD, 0x93, 0x26, 0x36, 0x3F, 0xF7, 0xCC, 0x34, 0xA5, 0xE5, 0xF1, 0x71, 0xD8, 0x31, 0x15,
    0x04, 0xC7, 0x23, 0xC3, 0x18, 0x96, 0x05, 0x9A, 0x07, 0x12, 0x80, 0xE2, 0xEB, 0x27, 0xB2, 0x75,
    0x09, 0x83, 0x2C, 0x1A, 0x1B, 0x6E, 0x5A, 0xA0, 0x52, 0x3B, 0xD6, 0xB3, 0x29, 0xE3, 0x2F, 0x84,
    0x53, 0xD1, 0x00, 0xED, 0x20, 0xFC, 0xB1, 0x5B, 0x6A, 0xCB, 0xBE, 0x39, 0x4A, 0x4C, 0x58, 0xCF,
    0xD0, 0xEF, 0xAA, 0xFB, 0x43, 0x4D, 0x33, 0x85, 0x45, 0xF9, 0x02, 0x7F, 0x50, 0x3C, 0x9F, 0xA8,
    0x51, 0xA3, 0x40, 0x8F, 0x92, 0x9D, 0x38, 0xF5, 0xBC, 0xB6, 0xDA, 0x21, 0x10, 0xFF, 0xF3, 0xD2,
    0xCD, 0x0C, 0x13, 0xEC, 0x5F, 0x97, 0x44, 0x17, 0xC4, 0xA7, 0x7E, 0x3D, 0x64, 0x5D, 0x19, 0x73,
    0x60, 0x81, 0x4F, 0xDC, 0x22, 0x2A, 0x90, 0x88, 0x46, 0xEE, 0xB8, 0x14, 0xDE, 0x5E, 0x0B, 0xDB,
    0xE0, 0x32, 0x3A, 0x0A, 0x49, 0x06, 0x24, 0x5C, 0xC2, 0xD3, 0xAC, 0x62, 0x91, 0x95, 0xE4, 0x79,
    0xE7, 0xC8, 0x37, 0x6D, 0x8D, 0xD5, 0x4E, 0xA9, 0x6C, 0x56, 0xF4, 0xEA, 0x65, 0x7A, 0xAE, 0x08,
    0xBA, 0x78, 0x25, 0x2E, 0x1C, 0xA6, 0xB4, 0xC6, 0xE8, 0xDD, 0x74, 0x1F, 0x4B, 0xBD, 0x8B, 0x8A,
    0x70, 0x3E, 0xB5, 0x66, 0x48, 0x03, 0xF6, 0x0E, 0x61, 0x35, 0x57, 0xB9, 0x86, 0xC1, 0x1D, 0x9E,
    0xE1, 0xF8, 0x98, 0x11, 0x69, 0xD9, 0x8E, 0x94, 0x9B, 0x1E, 0x87, 0xE9, 0xCE, 0x55, 0x28, 0xDF,
    0x8C, 0xA1, 0x89, 0x0D, 0xBF, 0xE6, 0x42, 0x68, 0x41, 0x99, 0x2D, 0x0F, 0xB0, 0x54, 0xBB, 0x16,
)

Rcon = (
    0x00, 0x01, 0x02, 0x04, 0x08, 0x10, 0x20, 0x40,
    0x80, 0x1B, 0x36, 0x6C, 0xD8, 0xAB, 0x4D, 0x9A,
    0x2F, 0x5E, 0xBC, 0x63, 0xC6, 0x97, 0x35, 0x6A,
    0xD4, 0xB3, 0x7D, 0xFA, 0xEF, 0xC5, 0x91, 0x39,
)


def text2matrix(text):
    matrix = []
    for i in range(16):
        byte = (text >> (8 * (15 - i))) & 0xFF
        if i % 4 == 0:
            matrix.append([byte])
        else:
            matrix[int(i / 4)].append(byte)
    return matrix

def change_key(master_key):
    round_keys = text2matrix(master_key)
    for i in range(4, 4 * 11):
        round_keys.append([])
        if i % 4 == 0:
            byte = round_keys[i - 4][0]        \
                     ^ Sbox[round_keys[i - 1][1]]  \
                     ^ Rcon[int(i / 4)]
            round_keys[i].append(byte)

            for j in range(1, 4):
                    byte = round_keys[i - 4][j]    \
                         ^ Sbox[round_keys[i - 1][(j + 1) % 4]]
                    round_keys[i].append(byte)
        else:
            for j in range(4):
                    byte = round_keys[i - 4][j]    \
                         ^ round_keys[i - 1][j]
                    round_keys[i].append(byte)

    return(round_keys)
"""
    Copyright (C) 2012 Bo Zhu http://about.bozhu.me

    Permission is hereby granted, free of charge, to any person obtaining a
    copy of this software and associated documentation files (the "Software"),
    to deal in the Software without restriction, including without limitation
    the rights to use, copy, modify, merge, publish, distribute, sublicense,
    and/or sell copies of the Software, and to permit persons to whom the
    Software is furnished to do so, subject to the following conditions:

    The above copyright notice and this permission notice shall be included in
    all copies or substantial portions of the Software.

    THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
    IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
    FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
    THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
    LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
    FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
    DEALINGS IN THE SOFTWARE.
"""

"""
    Modified by Guy Gogniat 2021
"""

# learnt from http://cs.ucsb.edu/~koc/cs178/projects/JT/aes.c
xtime = lambda a: (((a << 1) ^ 0x1B) & 0xFF) if (a & 0x80) else (a << 1)


def matrix2text(matrix):
    text = 0
    for i in range(4):
        for j in range(4):
            text |= (matrix[i][j] << (120 - 8 * (4 * i + j)))
    return text

def encrypt(plaintext, master_key):
        round_keys = change_key(master_key)
        plain_state = text2matrix(plaintext)
        add_round_key(plain_state, round_keys[:4])

        for i in range(1, 10):
            round_encrypt(plain_state, round_keys[4 * i : 4 * (i + 1)])

        sub_bytes(plain_state)
        shift_rows(plain_state)
        add_round_key(plain_state, round_keys[40:])

        return matrix2text(plain_state)


def add_round_key(s, k):
        for i in range(4):
            for j in range(4):
                s[i][j] ^= k[i][j]


def round_encrypt(state_matrix, key_matrix):
       sub_bytes(state_matrix)
       shift_rows(state_matrix)
       mix_columns(state_matrix)
       add_round_key(state_matrix, key_matrix)
       pass

def sub_bytes(s):
  "build the sub_bytes transformation"
  for i in range(4):
      for j in range(4):
          s[i][j] = Sbox[s[i][j]]
  pass

def shift_rows(s):
  "build the shift_rows transformation"
  s[0][1], s[1][1], s[2][1], s[3][1] = s[1][1], s[2][1], s[3][1], s[0][1]
  s[0][2], s[1][2], s[2][2], s[3][2] = s[2][2], s[3][2], s[0][2], s[1][2]
  s[0][3], s[1][3], s[2][3], s[3][3] = s[3][3], s[0][3], s[1][3], s[2][3]

def mix_single_column(a):
        # please see Sec 4.1.2 in The Design of Rijndael
        t = a[0] ^ a[1] ^ a[2] ^ a[3]
        u = a[0]
        a[0] ^= t ^ xtime(a[0] ^ a[1])
        a[1] ^= t ^ xtime(a[1] ^ a[2])
        a[2] ^= t ^ xtime(a[2] ^ a[3])
        a[3] ^= t ^ xtime(a[3] ^ u)


def mix_columns(s):
        for i in range(4):
            mix_single_column(s[i])
"""
    Copyright (C) 2012 Bo Zhu http://about.bozhu.me

    Permission is hereby granted, free of charge, to any person obtaining a
    copy of this software and associated documentation files (the "Software"),
    to deal in the Software without restriction, including without limitation
    the rights to use, copy, modify, merge, publish, distribute, sublicense,
    and/or sell copies of the Software, and to permit persons to whom the
    Software is furnished to do so, subject to the following conditions:

    The above copyright notice and this permission notice shall be included in
    all copies or substantial portions of the Software.

    THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
    IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
    FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
    THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
    LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
    FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
    DEALINGS IN THE SOFTWARE.
"""

"""
    Modified by Guy Gogniat 2021
"""

InvSbox = (
    0x52, 0x09, 0x6A, 0xD5, 0x30, 0x36, 0xA5, 0x38, 0xBF, 0x40, 0xA3, 0x9E, 0x81, 0xF3, 0xD7, 0xFB,
    0x7C, 0xE3, 0x39, 0x82, 0x9B, 0x2F, 0xFF, 0x87, 0x34, 0x8E, 0x43, 0x44, 0xC4, 0xDE, 0xE9, 0xCB,
    0x54, 0x7B, 0x94, 0x32, 0xA6, 0xC2, 0x23, 0x3D, 0xEE, 0x4C, 0x95, 0x0B, 0x42, 0xFA, 0xC3, 0x4E,
    0x08, 0x2E, 0xA1, 0x66, 0x28, 0xD9, 0x24, 0xB2, 0x76, 0x5B, 0xA2, 0x49, 0x6D, 0x8B, 0xD1, 0x25,
    0x72, 0xF8, 0xF6, 0x64, 0x86, 0x68, 0x98, 0x16, 0xD4, 0xA4, 0x5C, 0xCC, 0x5D, 0x65, 0xB6, 0x92,
    0x6C, 0x70, 0x48, 0x50, 0xFD, 0xED, 0xB9, 0xDA, 0x5E, 0x15, 0x46, 0x57, 0xA7, 0x8D, 0x9D, 0x84,
    0x90, 0xD8, 0xAB, 0x00, 0x8C, 0xBC, 0xD3, 0x0A, 0xF7, 0xE4, 0x58, 0x05, 0xB8, 0xB3, 0x45, 0x06,
    0xD0, 0x2C, 0x1E, 0x8F, 0xCA, 0x3F, 0x0F, 0x02, 0xC1, 0xAF, 0xBD, 0x03, 0x01, 0x13, 0x8A, 0x6B,
    0x3A, 0x91, 0x11, 0x41, 0x4F, 0x67, 0xDC, 0xEA, 0x97, 0xF2, 0xCF, 0xCE, 0xF0, 0xB4, 0xE6, 0x73,
    0x96, 0xAC, 0x74, 0x22, 0xE7, 0xAD, 0x35, 0x85, 0xE2, 0xF9, 0x37, 0xE8, 0x1C, 0x75, 0xDF, 0x6E,
    0x47, 0xF1, 0x1A, 0x71, 0x1D, 0x29, 0xC5, 0x89, 0x6F, 0xB7, 0x62, 0x0E, 0xAA, 0x18, 0xBE, 0x1B,
    0xFC, 0x56, 0x3E, 0x4B, 0xC6, 0xD2, 0x79, 0x20, 0x9A, 0xDB, 0xC0, 0xFE, 0x78, 0xCD, 0x5A, 0xF4,
    0x1F, 0xDD, 0xA8, 0x33, 0x88, 0x07, 0xC7, 0x31, 0xB1, 0x12, 0x10, 0x59, 0x27, 0x80, 0xEC, 0x5F,
    0x60, 0x51, 0x7F, 0xA9, 0x19, 0xB5, 0x4A, 0x0D, 0x2D, 0xE5, 0x7A, 0x9F, 0x93, 0xC9, 0x9C, 0xEF,
    0xA0, 0xE0, 0x3B, 0x4D, 0xAE, 0x2A, 0xF5, 0xB0, 0xC8, 0xEB, 0xBB, 0x3C, 0x83, 0x53, 0x99, 0x61,
    0x17, 0x2B, 0x04, 0x7E, 0xBA, 0x77, 0xD6, 0x26, 0xE1, 0x69, 0x14, 0x63, 0x55, 0x21, 0x0C, 0x7D,
)


def decrypt(ciphertext, master_key):
        round_keys = change_key(master_key)
        cipher_state = text2matrix(ciphertext)

        add_round_key(cipher_state, round_keys[40:])
        inv_shift_rows(cipher_state)
        inv_sub_bytes(cipher_state)

        for i in range(9, 0, -1):
            round_decrypt(cipher_state, round_keys[4 * i : 4 * (i + 1)])

        add_round_key(cipher_state, round_keys[:4])

        return matrix2text(cipher_state)


def round_decrypt(state_matrix, key_matrix):
    "call to the four round operations"
    add_round_key(state_matrix, key_matrix)
    inv_mix_columns(state_matrix)
    inv_shift_rows(state_matrix)
    inv_sub_bytes(state_matrix)
    pass

def inv_sub_bytes(s):
        "build the inv_sub_bytes transformation"
        for i in range(4):
            for j in range(4):
                s[i][j] = InvSbox[s[i][j]]
        pass

def inv_shift_rows(s):
        "build the inv_shift_rows transformation"
        s[1][1], s[2][1], s[3][1], s[0][1] = s[0][1], s[1][1], s[2][1], s[3][1]
        s[2][2], s[3][2], s[0][2], s[1][2] = s[0][2], s[1][2], s[2][2], s[3][2]
        s[3][3], s[0][3], s[1][3], s[2][3] = s[0][3], s[1][3], s[2][3], s[3][3]
        pass


def inv_mix_columns(s):
        # see Sec 4.1.3 in The Design of Rijndael
        for i in range(4):
            u = xtime(xtime(s[i][0] ^ s[i][2]))
            v = xtime(xtime(s[i][1] ^ s[i][3]))
            s[i][0] ^= u
            s[i][1] ^= v
            s[i][2] ^= u
            s[i][3] ^= v

        mix_columns(s)

In [ ]:
# @title Test file creation
filename = "test.txt"
with open(filename, "w") as file:
    file.write("This is a text file. You can do what you want with it.")

# Protocol 1: the Basic Key Transport Protocol with RSA 2048 and AES 128.

In [ ]:
# @title BOB and ALICE objects definitions
import os
def chunkstring(string, length):
    return (string[0+i:length+i] for i in range(0, len(string), length))
class BOB:
    def __init__(self):
      self.pub_key = None
      self.private_key = None
      self.key_name = None
      self.sym_key = None
    def generateKeyPair(self,key_name="BOB_KEY"):
      self.key_name = key_name
      try:
        makeKeyFiles(key_name, 2048)
      except SystemExit:
        pass
      finally:
        self.pub_key=readKeyFile(key_name+'_pubkey.txt')
        self.private_key=readKeyFile(key_name+'_privkey.txt')
    def receiveAndDecryptEncryptedKey(self, enc_key):
      keySize, n, d = self.private_key
      decryptedBlocks = decryptMessage(enc_key[1],enc_key[0], (n, d))
      self.sym_key = int(decryptedBlocks)
      print("Bob received and decrypted symmetric key:", hex(self.sym_key))
    def encryptMessage(self,message):
      print("Bob encrypts message: ", message)
      message = message.encode('utf-8')
      ciphertext = []
      for chunk in chunkstring(message, 16):
        plaintext = chunk.hex()
        plaintext = int(plaintext,16)
        ciphertext.append(encrypt(plaintext, self.sym_key))
      return ciphertext
class ALICE:
    def __init__(self):
      self.bob_pub_key = None
      self.sym_key = None
      self.encrypted_key = None
    def receivePublicKey(self,pub_key):
      #bob send his public key
      self.bob_pub_key = pub_key
      print("Alice received BOB public key:", pub_key)
    def generateSymKey(self):
      self.sym_key = int(os.urandom(16).hex(), 16)
      print("Alice generated symmetric key:", hex(self.sym_key))
    def encryptSymKey(self):
      message = self.sym_key
      message = str(message)
      length = len(message)
      keySize, n, e = self.bob_pub_key
      encryptedBlocks = encryptMessage(message, (n, e))
      self.encrypted_key = length, encryptedBlocks
      print("Alice encrypted symmetric key:", self.encrypted_key)
    def receiveAndDecryptMessage(self, enc_message):
      print("Alice receives encrypted message: ", enc_message)
      decrypted_aes_string = ""
      for block in enc_message:
        decrypted_aes = decrypt(block, self.sym_key)
        decrypted_aes_string += bytes.fromhex(format(decrypted_aes,'x')).decode('utf-8')
      print("Alice decrypts it: ",decrypted_aes_string)
      return decrypted_aes_string

In [ ]:
# @title Usage: Bob sends Alice his public key and Alice sends bob the session key
a = ALICE()
b = BOB()
# Bob generate his key pair if not existent
b.generateKeyPair()

# Bob sends his public key to Alice, that receives it
a.receivePublicKey(b.pub_key)
#Alice generate a symmetric key
a.generateSymKey()
a.encryptSymKey()

#Alice sends her public key to Bob, that receives it
b.receiveAndDecryptEncryptedKey(a.encrypted_key)


Alice received BOB public key: (2048, 65422146594804796970827594052837415808750651339936929509685909568402571284433001749856532872261272145302300826894524983973874546477646930839855085394189113609941588744702527622050219752863796976160684114325749101088929768542948250252934394270791180367134861774645327543868984578610987427863248427975859499222370502238384251047559878248894943207565418209426880311927620815939487653874429473763952089971168754483254697378201340447174289754932168221675727114490152039243108316401226085789109651091877233800362556631889383570802466017486015815130451568565318957741166984662651922854680367451213076951512635851534858721557746457057456367783157890571653482262132699191499334205624150528931627095667820859200594873804812532924592477090017626807519461957752014594325234171544364903279304142946899596262214098483179463359479051738989561946856887081016039470552428898133616045997608519767782505040612917954429190158737635219172367558007394612628060387986945411647663975549

In [ ]:
# @title Usage: Bob sends Alice an encrypted message
# Now alice and bob can switch to symmetric cryptography
f = open(filename, "r")
secret_message = f.read()
f.close()
# Bob sends alice the message
ciphertext = b.encryptMessage(secret_message)
decrypted = a.receiveAndDecryptMessage(ciphertext)


Bob encrypts message:  This is a text file. You can do what you want with it.
Alice receives encrypted message:  [16333589921559493666030252132905832054, 136647785335824575784693466889077909133, 45384997737233054734537301036340135845, 73012205526111620647406036205288115165]
Alice decrypts it:  This is a text file. You can do what you want with it.


#Protocol 2: The RSA Signature Protocol

In [ ]:
# @title BOB and ALICE objects definitions
from hashlib import sha256
class BOB:
    def __init__(self):
      self.pub_key = None
      self.private_key = None
      self.key_name = None
      self.digest = None
      self.signature = None
    def generateKeyPair(self,key_name="BOB_KEY"):
      self.key_name = key_name
      try:
        makeKeyFiles(key_name, 2048)
      except SystemExit:
        pass
      finally:
        self.pub_key=readKeyFile(key_name+'_pubkey.txt')
        self.private_key=readKeyFile(key_name+'_privkey.txt')
    def computeDigest(self, msg):
      self.digest = sha256(msg.encode('utf-8')).hexdigest()
      print("BOB computed digest:", self.digest)
    def signDigest(self):
      keySize, n, d = self.private_key
      length = len(self.digest)
      self.signature = length, encryptMessage(self.digest, (n, d))
      print("BOB signed digest:", self.signature)
class ALICE:
    def __init__(self):
      self.bob_pub_key = None
      self.bob_digest = None
      self.message = None
    def receivePublicKey(self,pub_key):
      #bob send his public key
      self.bob_pub_key = pub_key
      print("Alice received BOB public key:", pub_key)
    def receiveMessage(self,msg):
      print("Alice received message:", msg)
      self.message = msg
    def receiveSignature(self, signature):
      print("Alice received signature:", signature)
      keySize, n, e = self.bob_pub_key
      self.bob_digest = decryptMessage(signature[1], signature[0] , (n, e))
      print("Alice decrypts digest:", self.bob_digest)
    def verifySignature(self):
      digest = sha256(self.message.encode('utf-8')).hexdigest()
      print("Alice computes digest:", digest)
      return digest == self.bob_digest

In [ ]:
# @title Usage: Alice validates a message sent by Bob
a = ALICE()
b = BOB()

# BOB generates his key pair and sends his public key to ALICE
b.generateKeyPair()
a.receivePublicKey(b.pub_key)

# BOB computes the signature for message x and sends it to ALICE, along with the message itself

f = open(filename, "r")
x = f.read()
f.close()
b.computeDigest(x)
b.signDigest()

a.receiveMessage(x)
a.receiveSignature(b.signature)

if(a.verifySignature()):
  print("Signature is valid!")
else:
  print("Signature is not valid!")

a.receiveMessage("I am Paul!")
a.receiveSignature(b.signature)

if(a.verifySignature()):
  print("Signature is valid!")
else:
  print("Signature is not valid!")



Alice received BOB public key: (2048, 65422146594804796970827594052837415808750651339936929509685909568402571284433001749856532872261272145302300826894524983973874546477646930839855085394189113609941588744702527622050219752863796976160684114325749101088929768542948250252934394270791180367134861774645327543868984578610987427863248427975859499222370502238384251047559878248894943207565418209426880311927620815939487653874429473763952089971168754483254697378201340447174289754932168221675727114490152039243108316401226085789109651091877233800362556631889383570802466017486015815130451568565318957741166984662651922854680367451213076951512635851534858721557746457057456367783157890571653482262132699191499334205624150528931627095667820859200594873804812532924592477090017626807519461957752014594325234171544364903279304142946899596262214098483179463359479051738989561946856887081016039470552428898133616045997608519767782505040612917954429190158737635219172367558007394612628060387986945411647663975549

#Protocol 3: The Diffie–Hellman Key Exchange protocol

In [ ]:
import random
#Defining P and A constants

# maximum value for random key (2048bit safe prime)
# https://2ton.com.au/getprimes/random/2048
P=0x61d373e962dfa5f9d115835788076dfecce2ce15b75df96cc1e5b09d2648a0ae602c4c5a486556e0e162b5676672b877340ef14f89d0403d6279f659b0090937fc4d1164db4ec6fa713217b71752b577e3dd2b6214816d4d73f552f0e4868ba95cfdb19f28e398407bafc13f9271213890fb060670840202de4496cdad81cac6bae2bf9a69d0fa5242cb5bcd612e18898d199e604d516452347e450866d55a9bef72995010601ebfe11c8811e64dd18d0e17cb88829f213aba8786d49e4ee89329995e93095d86c28b074712f0f99ac5f53c71ffaaa1fd676e3bec15222b7e656a452628797d3e2b7978269877e0481c57401fb877a4a19e8727b8490244fe69

# We choose 5 because p ≡ 1 mod 4
A = 5 # base for exponentiation

# @title BOB and ALICE objects definitions
from hashlib import sha256
class BOB:
    def __init__(self):
      self.public_key = None
      self.private_key = None
      self.alice_public_key = None
      self.common_secret = None
      self.aes_key = None
    def generateKey(self):
      self.private_key = random.randint(1, P)
      print("Bob generates private key:", self.private_key)
    def computePublicKey(self):
      self.public_key = pow(A, self.private_key, P)
      print("Bob computes public key:", self.public_key)
    def receiveAlicePublicKey(self, key):
      self.alice_public_key = key
    def computeCommonSecret(self):
      self.common_secret = pow(self.alice_public_key,self.private_key,P)
      print("Bob computes common secret:", self.common_secret)
    def deriveKey(self):
      self.aes_key = int(hashlib.md5(str(self.common_secret).encode()).hexdigest(), 16)
      print("Bob derived secret key:", hex(self.aes_key))
    def encryptMessage(self,message):
      print("Bob encrypts message: ", message)
      message = message.encode('utf-8')
      ciphertext = []
      for chunk in chunkstring(message, 16):
        plaintext = chunk.hex()
        plaintext = int(plaintext,16)
        ciphertext.append(encrypt(plaintext, self.aes_key))
      return ciphertext
class ALICE:
    def __init__(self):
      self.public_key = None
      self.private_key = None
      self.bob_public_key = None
      self.common_secret = None
      self.aes_key = None
    def generateKey(self):
      self.private_key = random.randint(1, P)
      print("Alice generates private key:", self.private_key)
    def computePublicKey(self):
      print(A, self.private_key, P, pow(A, self.private_key, P))
      self.public_key = pow(A, self.private_key, P)
      print("Alice computes public key:", self.public_key)
    def receiveBobPublicKey(self, key):
      self.bob_public_key = key
    def computeCommonSecret(self):
      self.common_secret = pow(self.bob_public_key,self.private_key,P)
      print("Alice computes common secret:", self.common_secret)
    def deriveKey(self):
      self.aes_key = int(hashlib.md5(str(self.common_secret).encode()).hexdigest(), 16)
      print("Alice derived secret key:", hex(self.aes_key))
    def receiveAndDecryptMessage(self, enc_message):
      print("Alice receives encrypted message: ", enc_message)
      decrypted_aes_string = ""
      for block in enc_message:
        decrypted_aes = decrypt(block, self.aes_key)
        decrypted_aes_string += bytes.fromhex(format(decrypted_aes,'x')).decode('utf-8')
      print("Alice decrypts it: ",decrypted_aes_string)
      return decrypted_aes_string

In [ ]:
# @title Usage: Alice and Bob exchange keys
a = ALICE()
b = BOB()

# Key generation
a.generateKey()
b.generateKey()

# Public key computation
a.computePublicKey()
b.computePublicKey()

# Key exchange
a.receiveBobPublicKey(b.public_key)
b.receiveAlicePublicKey(a.public_key)

# Key agreement (Key is longer than needed)
a.computeCommonSecret()
b.computeCommonSecret()

# Key derivation (Applying hash function to obtain 128 bits)
a.deriveKey()
b.deriveKey()

Alice generates private key: 24167220667517313305876027743347371901104007664237621155442728829018689997003982349932642222668477301643311443695276821501303358623773564346756911919327705362305270501850465527003307027279400703849688464883572785261365289218743561920327034729823933754299366417228839125996273921107437634430129217990764388117319193238104492885085010421781522240458683944024070909634632051050050997160666368657250399734366757547952885954402219668658122230601805231501223292957361949689788597940961679888605483247614813154059185039516160028639401341783869652267305839682935938995771422514423076714414780339077463195543043229841325914
Bob generates private key: 69147062732883587927886119772186082766438651293195441464464810341531016631691852878109152596953149819253919607392929275032418497000964555495829276507629241765870619605437459097262901755317066212545611337923772431585472124615326810722407915373967882055090186395380444409257277972982413451265241237216726481799443076969397024448362

In [ ]:
# @title Usage: Bob send an encrypted message to Alice
# Now alice and bob can switch to symmetric cryptography
f = open(filename, "r")
secret_message = f.read()
f.close()
# Bob sends alice the message
ciphertext = b.encryptMessage(secret_message)
decrypted = a.receiveAndDecryptMessage(ciphertext)

Bob encrypts message:  This is a text file. You can do what you want with it.
Alice receives encrypted message:  [170486179078025055603188525677476902738, 91869743615486856191373705854381767106, 54901028453814332282446699667170211795, 73528080354927536621274088181852889343]
Alice decrypts it:  This is a text file. You can do what you want with it.
